In [1]:
!pip install -U "autogluon.tabular" pandas scikit-learn

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 699.0 kB/s eta 0:00:16
   - -------------------------------------- 0.5/11.3 MB 699.0 kB/s eta 0:00:16
   - -------------------------------------- 0.5/11.3 MB 699.0 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/11.3 MB 516.0 kB/s eta 0:00:21
   --- ------------------------------------ 1.0/11.3 MB 592.2 kB/s eta 0:00:18
   --- ------------------------------------ 1.0/11.3 MB 592.2 kB/s eta 0:00:18
   ---- ----------------------------------- 1.3/11.3 MB 633.2 kB/s eta 0:00:16
   ---- ----------------------------------- 1.3/11.3 MB 633.2 kB/s eta 0:00:16
   ----- ---------------------------------- 1.6/11.3 MB 655.4 kB/s eta 0:00:15
   ------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
pycaret 3.3.2 requires pandas<2.2.0, but you have pandas 2.3.3 which is incompatible.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.3.3 which is incompatible.
sktime 0.26.0 requires scikit-learn<1.5.0,>=0.24, but you have scikit-learn 1.7.2 which is incompatible.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from autogluon.tabular import TabularPredictor

In [3]:
# 1) Load
df = pd.read_csv("ckd.csv")
df.head()

,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
0,1,71,0,0,0,2,31.069414,1,5.128112,1.676220,...,7.556302,76.076800,0,0,1,1.018824,4.966808,9.871449,1,Confidential
1,2,34,0,0,1,3,29.692119,1,18.609552,8.377574,...,6.836766,40.128498,0,0,0,3.923538,8.189275,7.161765,1,Confidential
2,3,80,1,1,0,1,37.394822,1,11.882429,9.607401,...,2.144722,92.872842,0,1,1,1.429906,7.624028,7.354632,1,Confidential
3,4,40,0,2,0,1,31.329680,0,16.020165,0.408871,...,7.077188,90.080321,0,0,0,3.226416,3.282688,6.629587,1,Confidential
4,5,43,0,1,1,2,23.726311,0,7.944146,0.780319,...,3.553118,5.258372,0,0,1,0.285466,3.849498,1.437385,1,Confidential


In [4]:
# 2) Drop unused columns
df = df.drop(columns=["PatientID", "DoctorInCharge"], errors="ignore")

In [5]:
# 3) Target
target = "Diagnosis"

In [6]:
# 4) Split
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42,
    stratify=df[target] if df[target].nunique() > 1 else None
)

In [7]:
# 5) AutoGluon AutoML
predictor = TabularPredictor(
    label=target,
    problem_type="binary",     # use "multiclass" if more than 2 classes
    eval_metric="f1",          # better than accuracy for imbalance
    path="autogluon_ckd_model" # folder to save models
).fit(
    train_data=train_df,
    presets="best_quality",    # "medium_quality_faster_train" if you want faster
    time_limit=300             # seconds
)

Verbosity: 2 (Standard Logging)
	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Pytorch Version:    Can't import torch
CUDA Version:       Can't get cuda version from torch
Memory Avail:       0.87 GB / 7.69 GB (11.3%)
Disk Space Avail:   18.85 GB / 163.21 GB (11.5%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data

In [8]:
# 6) Predict + Evaluate
y_true = test_df[target]
y_pred = predictor.predict(test_df.drop(columns=[target]))

print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

Accuracy: 0.927710843373494
              precision    recall  f1-score   support

           0       0.62      0.30      0.40        27
           1       0.94      0.98      0.96       305

    accuracy                           0.93       332
   macro avg       0.78      0.64      0.68       332
weighted avg       0.91      0.93      0.92       332



In [9]:
# 7) Leaderboard (all tried models)
print(predictor.leaderboard(test_df, silent=True))

                     model  score_test  score_val eval_metric  pred_time_test  \
0          LightGBM_BAG_L1    0.963665   0.968588          f1        0.092565   
1      WeightedEnsemble_L2    0.963665   0.968588          f1        0.148163   
2           XGBoost_BAG_L1    0.963434   0.968153          f1        0.181739   
3          LightGBM_BAG_L2    0.962145   0.966667          f1        1.750929   
4        LightGBMXT_BAG_L2    0.961538   0.970812          f1        1.347458   
5      WeightedEnsemble_L3    0.961538   0.970812          f1        1.353236   
6  RandomForestGini_BAG_L1    0.959119   0.958710          f1        0.258703   
7  RandomForestEntr_BAG_L1    0.959119   0.957957          f1        0.296653   
8    ExtraTreesGini_BAG_L1    0.957614   0.957581          f1        0.200327   
9    ExtraTreesEntr_BAG_L1    0.957614   0.957581          f1        0.223320   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.035499   78.79282

In [15]:
# 8) Best model info + save path
lb = predictor.leaderboard(test_df, silent=True)
best_model = lb.iloc[0]["model"]
print("Best model:", best_model)

Best model: LightGBM_BAG_L1


In [17]:
info = predictor.info()
print(info["model_info"][best_model])

{'name': 'LightGBM_BAG_L1', 'model_type': 'StackerEnsembleModel', 'problem_type': 'binary', 'eval_metric': 'f1', 'stopping_metric': 'f1', 'fit_time': 78.79282879829407, 'num_classes': 2, 'quantile_levels': None, 'predict_time': 0.035498857498168945, 'val_score': 0.9685884691848906, 'hyperparameters': {'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True, 'stratify': 'auto', 'bin': 'auto', 'n_bins': None, 'vary_seed_across_folds': False, 'model_random_seed': 0}, 'hyperparameters_user': {}, 'hyperparameters_fit': {}, 'hyperparameters_nondefault': [], 'ag_args_fit': {'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'd

In [20]:
info = predictor.info()

try:
    best_model = predictor.get_model_best_name()
except Exception:
    best_model = predictor.leaderboard(test_df, silent=True).iloc[0]["model"]

model_info = info["model_info"][best_model]

print("Best model:", best_model)
print("Bagged child model type:", model_info["bagged_info"]["child_model_type"])
print("Child hyperparameters:", model_info["bagged_info"]["child_hyperparameters"])

# num_boost_round per fold
children = model_info["children_info"]
boost_rounds = {k: v.get("hyperparameters_fit", {}).get("num_boost_round") for k, v in children.items()}
print("num_boost_round per fold:", boost_rounds)

# optional: average trees
vals = [v for v in boost_rounds.values() if v is not None]
print("Average num_boost_round:", sum(vals)/len(vals))

Best model: LightGBM_BAG_L1
Bagged child model type: LGBModel
Child hyperparameters: {'learning_rate': 0.05, 'seed': 0}
num_boost_round per fold: {'S1F1': 32, 'S1F2': 112, 'S1F3': 17, 'S1F4': 30, 'S1F5': 40, 'S1F6': 53, 'S1F7': 314, 'S1F8': 136}
Average num_boost_round: 91.75


## What best model actually is

* **Best model name:** `LightGBM_BAG_L1`
* **Model type:** `StackerEnsembleModel` (but with **0 base models** → it’s essentially a *bagged* LightGBM ensemble, not a stacked ensemble)
* **Bagging:** 8 child models (`S1F1 … S1F8`)
* **Metric:** `f1`
* **Child model type:** `LGBModel` (LightGBM)

## Best model hyperparameters (important ones)

### Ensemble-level (bagging settings)

From the parent (`LightGBM_BAG_L1` → `hyperparameters`):

* `save_bag_folds: True`
* `vary_seed_across_folds: False`
* `model_random_seed: 0`
* `use_orig_features: True`

### LightGBM (child model) parameters

From `bagged_info["child_hyperparameters"]`:

* `learning_rate = 0.05`
* `seed = 0`

And per-fold fitted parameter (varies):

* `num_boost_round` (number of trees) differs per fold, e.g. `S1F1: 32`, `S1F7: 314`, etc.

So the key “best model parameters” you have are mainly:

* **learning_rate = 0.05**
* **num_boost_round = (varies per fold)**

AutoGluon didn’t store many other LightGBM params here because it used its internal defaults + early stopping.